In [1]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
path_to_project = path_to_this_notebook[:path_to_this_notebook.find('note')]
sys.path.append(path_to_project)

from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle


In [2]:
# Load data
full_df = pd.DataFrame()
columns_to_take = ['TIMESTAMP', 'InvPDC_kW_Avg', 'InvPAC_kW_Avg', 'PwrMtrP_kW_Avg']
p_cols = ['InvPDC_kW_Avg', 'InvPAC_kW_Avg', 'PwrMtrP_kW_Avg']
path_to_data = path_to_project + '/data/pvdata.nist.gov/'
for year in ['2015', '2016', '2017']:
    for month in sorted(os.listdir(path_to_data + year + '/')):
        for day in sorted(os.listdir(path_to_data + year + '/' + month + '/')):
            path = path_to_data + year + '/' + month + '/' + day
            df = pd.read_csv(path)[columns_to_take]
            full_df = pd.concat([full_df, df])
# Create date and time columns         
date = full_df['TIMESTAMP'].apply(lambda x: x[:10])
time = full_df['TIMESTAMP'].apply(lambda x: x[11:16])
full_df['date'] = date
full_df['time'] = time
full_df = full_df.drop(['TIMESTAMP'], axis=1)
full_df.head()

,InvPDC_kW_Avg,InvPAC_kW_Avg,PwrMtrP_kW_Avg,date,time
0,0.0,0.0,0.0,2015-01-01,00:00
1,0.0,0.0,0.0,2015-01-01,00:01
2,0.0,0.0,0.0,2015-01-01,00:02
3,0.0,0.0,0.0,2015-01-01,00:03
4,0.0,0.0,0.0,2015-01-01,00:04


In [3]:
group_th = 60

# Determine bad indices which have either nan or large negative value
df_processed = full_df.copy()
for c in p_cols:
    df_processed[c] = full_df[c].apply(lambda x: max(0, x) if x >= -1 else np.nan)
df_processed = df_processed.reset_index(drop=True)

inds_na1 = np.where(df_processed['InvPDC_kW_Avg'].isna())[0]
inds_na2 = np.where(df_processed['InvPAC_kW_Avg'].isna())[0]
inds_na3 = np.where(df_processed['PwrMtrP_kW_Avg'].isna())[0]
bad_inds = set(inds_na1) | set(inds_na2) | set(inds_na3)
bad_inds = sorted(list(bad_inds))
print(len(bad_inds), len(bad_inds) / len(df_processed))

# Group indexes such that group is [i, i+1, i+2, i+g]
bad_groups_raw = []
prev_i = None
current_group = []

for i in bad_inds:
    if len(current_group) == 0:
        current_group.append(i)
    else:
        if i - 1 == current_group[-1] and df_processed['date'].iloc[i] ==  df_processed['date'].iloc[i - 1]:
            current_group.append(i)
        else:
            bad_groups_raw.append(list(current_group))
            current_group = [i]
            
if len(current_group):
    bad_groups_raw.append(list(current_group))
    
# For groups of nan smaller than group_th, we simply interpolate them. We use large value of group_th, 
# but it's fine since ther are not that many large groups. We also add multiplicative noise during interpolation
bad_groups = [] 
for group in bad_groups_raw:
    if len(group) > group_th:
        bad_groups.append(list(group))
    else:
        for c in p_cols:
            val_start = df_processed.iloc[group[0] -1][c]
            val_end = df_processed.iloc[group[-1] + 1][c]
            new_group_vals = np.linspace(val_start, val_end, len(group) + 2)[1: -1]
            for i, val in zip(group, new_group_vals * np.random.normal(1, 0.05, size=len(group))):
                df_processed.loc[i, c] = val
        

10309 0.006531959651257096


In [4]:
# Update bad inds 
bad_inds = set()
for gr in bad_groups:
    for i in gr:
        bad_inds.add(i)
bad_inds = list(bad_inds)
len(bad_inds), len(bad_groups)

(9132, 10)

In [5]:
# Compute what dates still have nans
dates_to_na = defaultdict(lambda: 0)
dates_to_na_inds = defaultdict(list)
for i in bad_inds:
    date = df_processed['date'].iloc[i]
    dates_to_na[date] += 1
    dates_to_na_inds[date].append(i)


In [6]:
dates_to_na, len(dates_to_na)

(defaultdict(<function __main__.<lambda>()>,
             {'2017-10-21': 1440,
              '2017-10-22': 1440,
              '2017-10-23': 753,
              '2015-06-14': 488,
              '2015-06-15': 438,
              '2017-06-15': 81,
              '2017-06-02': 1025,
              '2017-06-03': 1440,
              '2017-06-04': 1018,
              '2017-10-20': 1009}),
 10)

In [7]:
# Simply drop all dates with nans
dates_to_drop = dates_to_na.keys()
mask = ~df_processed['date'].isin(dates_to_drop)
df_processed = df_processed[mask].reset_index(drop=True)

In [8]:
metadata_dict ={'solar_rated_power': 243, }
dates = df_processed['date'].unique()
metadata_dict['dates'] = dates

In [9]:
with open(path_to_data + 'metadata_dict.pickle', 'wb') as f:
    pickle.dump(metadata_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# Save
df_processed.to_csv(path_to_data + 'processed_data.csv', index=None)